In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.append('/home/matt/readout/host/')
import ali_offline_demod as dm
import imp
import glob
import pickle
from scipy import spatial
import h5py
from scipy.fft import fft, ifft, fftfreq
from copy import deepcopy
from scipy.interpolate import CubicSpline, interp1d
from scipy.optimize import curve_fit
from tqdm.notebook import tqdm
import pandas as pd
from tqdm.notebook import tqdm as tqdm_notebook

/tmp/ipykernel_521167/2747086407.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [2]:
#first we need a function that reads the appropriate channel file and returns its information

def read_channel_file(channel_file):
    channel_standard = pd.read_csv(channel_file,sep=',')
    channel_nums = channel_standard['Channel Number'].to_numpy(int)
    channel_freqs = channel_standard['Frequency'].to_numpy(float)
    bias_lines = channel_standard['Bias Line'].to_numpy(int)
    optically_active = channel_standard['Optically Active'].to_numpy(int)
    selected_channels = channel_standard['Selected'].to_numpy(int)
    
    return channel_standard, channel_nums, channel_freqs, bias_lines, optically_active, selected_channels
       
    
channel_standard, channel_nums, channel_freqs, bias_lines, optically_active, selected_channels = read_channel_file('/home/matt/readout/host/channel_standards/LO_4250MHz.csv')

In [92]:
#now we need to create a new function that does the channel setup; this will be identical to what we already do in offline demod,
#but it will only run once at the beginning of the live stream

imp.reload(dm)

def channel_setup(init_directory, channel_standard_file):
        
    initial_lo_sweep_path = dm.find_file(init_directory, 'lo_sweep_initial')
    targeted_lo_sweep_path = dm.find_file(init_directory, 'lo_sweep_targeted_2')
    tone_freqs_path = dm.find_file(init_directory, 'freq_list_lo_sweep_targeted_1')
    
    channel_standard, channel_nums, channel_freqs, bias_lines, optically_active, selected_channels = read_channel_file(channel_standard_file)
    
    #standard channels to use
    active_channels = np.argwhere(selected_channels == 1)
    use_standard_channels = np.reshape(active_channels,(1,active_channels.shape[0]))[0]    
    
    #loading up tone init
    initial_lo_sweep=np.load(initial_lo_sweep_path) #find initial lo sweep file
    targeted_lo_sweep=np.load(targeted_lo_sweep_path) #find targeted sweep file
    tone_freqs=np.load(tone_freqs_path) #find tone freqs
    
    #converting standard channels to tone init channels
    match = dm.match_freqs(tone_freqs,channel_freqs[use_standard_channels],dist=1e5)
    #print(match)
    use_current_init_channels = np.array([item[0] for item in match])
    use_current_init_channels = use_current_init_channels[0:int(len(use_current_init_channels)/2)]
    #print(use_current_init_channels)
    
    
    #frequencies of channels    
    current_freqs = channel_freqs[use_current_init_channels]
    
    #compute delay region
    print('looking for delay region')
    delay_region_start, delay_region_stop = dm.find_freqs_cable_delay_subtraction(initial_lo_sweep,0.98,10000)
    print(f'start = {delay_region_start}')
    print(f'stop = {delay_region_stop}')
    
    #measure cable delay
    delays = dm.measure_delay_test_given_freq(initial_lo_sweep,delay_region_start,delay_region_stop,plot=False)
    
    print(f'delay: {np.median(delays)}')
    
    #remove cable delay
    targeted_lo_sweep_rm=dm.remove_delay_live(targeted_lo_sweep,
                                           np.median(delays),
                                           channels=use_current_init_channels)
    
    #measure circle parameters
    calibration=dm.measure_circle_live(targeted_lo_sweep_rm,
                                       tone_freqs,
                                       channels=use_current_init_channels) #finds circle center and initial phase for every channel
    
    print(calibration[0])
    
    return np.median(delays), calibration, use_current_init_channels, current_freqs
    
    

delay, calibration, channels, channel_freqs = channel_setup('/home/matt/alicpt_data/tone_initializations/fcenter_4250.0_20240216162215/', 
                                                            '/home/matt/readout/host/channel_standards/LO_4250MHz.csv')


#print(channels)
    


/home/matt/readout/host/ali_offline_demod.py:1619: ComplexWarning: Casting complex values to real discards the imaginary part
  idups_ids=[fall[x,1].astype(int) for x in idups_redu]
/home/matt/readout/host/ali_offline_demod.py:1625: ComplexWarning: Casting complex values to real discards the imaginary part
  intermediate_plotting_limits[0] = t[0]
/home/matt/readout/host/ali_offline_demod.py:1627: ComplexWarning: Casting complex values to real discards the imaginary part
  intermediate_plotting_limits[1] = t[-1]


looking for delay region
start = 4243240257.8125
stop = 4256008812.5
delay: 6.430838031037784e-08
[-2.67531070e+03+158.07501974j  1.82143861e+03  +0.j
 -4.10460336e-01  +0.j        ]


/home/matt/readout/host/ali_offline_demod.py:1005: RuntimeWarning: divide by zero encountered in log10
  x=np.append(x,test_sweep[0,:].real)


In [93]:
#now we need the part of the code that takes a chunk of data and demods it

%matplotlib qt

imp.reload(dm)

def demod_routine_live(ts_file, channels, channel_freqs, delay, calibration, cadence = 3, f_sawtooth 15, data_rate = 2**19):
    
    ts_fr,Is_fr,Qs_fr=dm.read_data_live(ts_file,
                                        channels=channels)
        
    fs=512e6/data_rate  
    
    truncate = cadence * fs
    ts_fr = np.arange(Is_fr.shape[1])/fs
    
    
    ts_fr = ts_fr[-truncate:]
    Is_fr = Is_fr[-truncate:]
    Qs_fr = Qs_fr[-truncate:]   
    
    IQ_stream_rm=dm.remove_delay_timestream(Is_fr+1j*Qs_fr,channel_freqs,delay)
    
    data_cal=dm.get_phase(IQ_stream_rm,calibration)   
        
    t_start=0
    t_stop=10
        
    t0 = np.array([dm.mea_reset_t0(ts_fr[int(fs)*t_start:int(fs)*t_stop],
                                   data_cal[current_channel,int(fs)*t_start:int(fs)*t_stop],
                                   f_sawtooth,plot=False)
                   for current_channel in range(len(data_cal))])

    t0_med = np.nanmedian(t0)
    
    start_idx = dm.find_nearest_idx(ts_fr-ts_fr[0], t0_med)
    
    channel_count = 0
    for chan in tqdm_notebook(range(data_cal.shape[0])):
    
        t_demod, data_demod, reset_indices = dm.demodulate_with_fft(t=ts_fr,
                                                                     sig=data_cal[chan],
                                                                     start_index=start_idx,                                                                      
                                                                     f_fr=f_sawtooth,
                                                                     phase_units='nPhi0',
                                                                     correct_phase_jumps=False,
                                                                     phase_jump_threshold=0,
                                                                     plot_demod = False,
                                                                     plot_demod_title=None,
                                                                     intermediate_plotting_limits=[None,None],
                                                                     plot_chunking_process = False,
                                                                     plot_fft = False,
                                                                     plot_fft_no_dc = False,
                                                                     plot_limited_fft = False,
                                                                     plot_fit = False,
                                                                     plot_vectors = False)
        
        if ch_count == 0:
            data_demods = data_demod
        else:
            data_demods = np.vstack([data_demods, np.array(data_demod)])
        t_demods = t_demod
        ch_count += 1
        
    return t_demod, data_demod
    
    
demod_routine_live('/home/matt/alicpt_data/time_streams/ts_toneinit_fcenter_4250.0_20240216162215_t_20240216163432.hd5',
                   channels,
                   channel_freqs,
                   delay,
                   calibration,
                   488,
                   15)


    





#print(delay)
#print(calibration)
"""
    
    
    #compute delay region
    print('looking for delay region')
    delay_region_start, delay_region_stop = find_freqs_cable_delay_subtraction(initial_lo_sweep,0.98,10000)
    print(f'start = {delay_region_start}')
    print(f'stop = {delay_region_stop}')
    
    #measure cable delay
    delays = measure_delay_test_given_freq(initial_lo_sweep,delay_region_start,delay_region_stop,plot=False)
    
    #channels to look at
    use_channels = np.reshape(np.argwhere(selected_channels == 1),(1,selected_channels.shape[0]))[0]
    
    #remove cable delay
    targeted_lo_sweep_rm=remove_delay_live(targeted_lo_sweep,
                                           np.median(delays),
                                           channels=s
    
    targeted_lo_sweep_rm=remove_delay(targeted_lo_sweep,
                                      np.median(delays),
                                      channels=channels,
                                      start_channel=start_channel,
                                      stop_channel=stop_channel)
    
    
    
    #measure circle parameters
    calibration=measure_circle_allch(targeted_lo_sweep_rm,
                                     tone_freqs,
                                     channels=channels,
                                     start_channel=start_channel,
                                     stop_channel=stop_channel) #finds circle center and initial phase for every channel
    
    #calibrate time stream
    data_cal=get_phase(IQ_stream_rm,calibration)
    
    #find nphi_0
    t_start=0
    t_stop=10

    n_phi0 = find_n_phi0(ts_fr[488*t_start:488*t_stop],data_cal[:,488*t_start:488*t_stop],f_sawtooth,plot=False)  #discard the first few seconds
    print(f'n_phi0: {n_phi0}')
    
    #find t0
    t0_array = np.array([])
    for current_channel in range(len(data_cal)):
        t0 = mea_reset_t0(ts_fr[488*t_start:488*t_stop],data_cal[current_channel,488*t_start:488*t_stop],f_sawtooth,plot=False)
        #ts_freq = 1/np.nanmedian(np.diff(ts_fr))
        #t0 = mea_reset_t0(ts_fr[ts_freq*t_start:ts_freq*t_stop],data_cal[current_channel,ts_freq*t_start:ts_freq*t_stop],f_sawtooth,plot=False)
        t0_array = np.append(t0_array,t0)

    t0_med = np.nanmedian(t0_array)
    
    return delays
"""

#IQ_stream_rm=remove_delay_timestream(Is_fr+1j*Qs_fr,tone_freqs,np.median(delays))
    

[ 22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39
  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57
  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75
  76  77  78  79  84  85  86  87  94  95  96  99 100 101 102 103 104 105
 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123
 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145
 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163
 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181
 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199
 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217
 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235
 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252]
[[ 0.16738983  0.18102925  0.18651077 ...  0.20217602  0.19909864
   0.18656007]
 [-1.92559804 -0.91326364 -0.07306417 ...  0.3

  0%|          | 0/215 [00:00<?, ?it/s]

"\n    \n    \n    #compute delay region\n    print('looking for delay region')\n    delay_region_start, delay_region_stop = find_freqs_cable_delay_subtraction(initial_lo_sweep,0.98,10000)\n    print(f'start = {delay_region_start}')\n    print(f'stop = {delay_region_stop}')\n    \n    #measure cable delay\n    delays = measure_delay_test_given_freq(initial_lo_sweep,delay_region_start,delay_region_stop,plot=False)\n    \n    #channels to look at\n    use_channels = np.reshape(np.argwhere(selected_channels == 1),(1,selected_channels.shape[0]))[0]\n    \n    #remove cable delay\n    targeted_lo_sweep_rm=remove_delay_live(targeted_lo_sweep,\n                                           np.median(delays),\n                                           channels=s\n    \n    targeted_lo_sweep_rm=remove_delay(targeted_lo_sweep,\n                                      np.median(delays),\n                                      channels=channels,\n                                      start_channel=star